# gu_toolkit: Efficient Guide, Compendium, and Applications

This notebook is an opinionated, practical guide for everyday toolkit use. It is organized for **fast onboarding** and **copy/paste utility**.

## How to use this notebook
1. Run the setup cell.
2. Jump to the section you need (quickstart, recipes, applications).
3. Copy a pattern and adapt it.

---

## Contents
- Quickstart
- Frequently used APIs (compendium)
- Parameter and numeric-expression patterns
- Introspection and debugging helpers
- Cool applications (interactive demos)


In [ ]:
# Find where the package root is
import sys
from pathlib import Path

try:
    _start = Path(__file__).resolve().parent
except NameError:
    _start = Path.cwd().resolve()

_pkg_root = _start
while _pkg_root != _pkg_root.parent and not (_pkg_root / "__init__.py").exists():
    _pkg_root = _pkg_root.parent
sys.path.insert(0, str(_pkg_root.parent))


In [ ]:
# Setup: Import toolkit
from gu_toolkit import *

# 1) Quickstart

## Minimal interactive figure

In [ ]:
fig = Figure(x_range=(-8, 8))
display(fig)
# TODO: Figure should autodisplay on creation unless DoNotDisplay=True is passed (or maybe a better worded name for the parameter)
with fig:
    set_title("Quickstart: Interactive sine")
    plot(a * sin(b * x), x, id="wave", label=r"$a \sin(b x)$")


## Figure with some configuration

In [ ]:
fig = Figure(x_range=(-3 * pi, 3 * pi))
display(fig)

with fig:
    set_title("Quickstart: Interacting sine waves")
    plot(
        sin(b[1] * x),
        x,
        id="wave1",
        label=r"$\sin(b_1x)$",
        color="blue",
        opacity=0.075,
        thickness=5,
    )
    plot(
        sin(b[2] * x),
        x,
        id="wave2",
        label=r"$\sin(b_2x)$",
        color="green",
        alpha=0.075,
        width=5,
    )
    # alpha is an alias for opacity, width is an alias for thickness
    plot(
        sin(b[1] * x) * sin(b[2] * x),
        x,
        id="product_wave",
        label=r"$\sin(b_1x) \sin(b_2x)$",
        color="red",
        opacity=1,
    )
    # color can be specified as a name or a hex value
    parameter(b[1], value=1, min=1, max=20, step=1)
    parameter(b[2], value=2, min=1, max=20, step=1)

## Saving Figure code

Make sure to run the code to create the figure. Play around with it and decide on specific values of the parameters. Later `figure.to_code()` can be used to print out the code that will generate the same figure

In [ ]:
print(fig.code)
# The generated snippet captures the current figure, parameters, and traces so it can be reused in another notebook.
# Use fig.to_code() for additional export options when you need a more customized script.


# BUG: by default use, from gu_toolkit import * . In this case you should use the sympy methods directly without the sp. prefix
# However, leave an option to have minimal imports

# BUG: Instead of defining symbols using sp.symbols gu_toolkit offers SymbolFamily and FunctionFamily that allow doing b[1] and b[2]
# once b is a member of said family.
# instead of defining b_1 etc manually. Prefer that but allow, with a config option to use standard sympy symbols.

# BUG: do not specify default values that were not given at call time e.g. y_range and sampling_points in this case. This is not easy, and requires some thought.

# BUG: some values were specified as sympy expressions (3*pi) before numpification. We should figure out a non-fragile way to remember this and record that expression rather than the numeric value.
# This requires a significant redesign, though.

## Plotting a purely numeric function or a python callable

The callable-first API supports plain Python functions as long as they can evaluate over NumPy arrays.


In [ ]:
fig_callable = Figure(x_range=(-6, 6), y_range=(-1.5, 1.5), sampling_points=500)
display(fig_callable)

with fig_callable:
    set_title("Numeric callable: damped oscillation")

    def damped_wave(x):
        return np.exp(-0.15 * x**2) * np.cos(3 * x)

    # For single-variable callables, pass the plotting symbol as the second argument.
    plot(damped_wave, x, id="damped", color="teal", label=r"$\exp(-0.15x^2) \cos(3x)$")

The callable-first API accepts a Python function as the first argument and the plotting symbol as the second argument.

This pattern is useful when your model is already written numerically (NumPy/JAX style) and you still want gu_toolkit slider and layout features.


In [ ]:
fig_callable_params = Figure(x_range=(-8, 8), y_range=(-2.5, 2.5), sampling_points=600)
display(fig_callable_params)

with fig_callable_params:
    set_title("Callable with explicit vars + sliders")
    # Use NumPy math inside numeric callables so array inputs evaluate correctly.
    plot(
        lambda x, A, k: A * np.sin(k * x),
        x,
        vars=(x, A, k),
        id="param-wave",
        label=r"$A*\sin(k x)$",
        color="orange",
    )
    # For multi-argument callables, declare vars=(...) so slider-backed parameters map explicitly.
    plot(
        lambda x, A, k: A + k * x,
        x,
        vars={"x": x, "A": A, "k": k[1]},
        id="param-line",
        label=r"$A+k_1 x$",
        color="blue",
    )
    parameter(A, value=1.2, min=0.0, max=2.5, step=0.1)
    parameter(k, value=1.0, min=0.5, max=4.0, step=0.1)

Tip: for numeric callables, prefer NumPy functions (for example, <code>np.sin</code>) rather than SymPy functions (such as <code>sin</code>).

SymPy functions expect symbolic inputs, while callable plotting evaluates on NumPy arrays during rendering.


## Quickstart: adding non-trivial info cards

Info cards can mix static HTML with dynamic callback segments that re-evaluate on render and slider changes.
Use this pattern to show diagnostics, derived metrics, and runtime context next to your plot.


Info cards let you attach rich, always-visible context to the figure panel. Use them for interpretation notes, formulas, and operating instructions so users can understand controls without leaving the notebook.

Practical guidance:
- Keep one card for *how to interact* (which slider controls what).
- Keep one card for *what to look for* (expected shape changes, limits, or invariants).
- Prefer short HTML snippets (`<b>`, `<code>`, `<br>`) so cards remain compact in narrow sidebars.


In [ ]:
fig_info = Figure(x_range=(-2 * sp.pi, 2 * sp.pi), y_range=(-3, 3), sampling_points=600)
display(fig_info)

with fig_info:
    set_title("Quickstart: plot + diagnostics info cards")
    plot(a * sin(b * x), x, id="main-wave", label=r"$a\sin(bx)$", color="royalblue")
    parameter(a, value=1.0, min=0.2, max=2.0, step=0.1)
    parameter(b, value=1.0, min=0.5, max=4.0, step=0.1)

    info(
        """
    <b>Tips</b><br>
    • Move <code>a</code> to scale amplitude.<br>
    • Move <code>b</code> to change frequency.<br>
    """,
        id="usage-tips",
    )

    def diagnostics_card(fig, ctx):
        p = fig.params.snapshot()
        amp = p[a]
        freq = p["b"]
        period = (2 * np.pi / freq) if freq else float("inf")
        est_energy = 0.5 * amp**2
        return (
            f"<b>Wave diagnostics</b><br>"
            f"reason: <code>{ctx.reason}</code><br>"
            f"a={amp:.2f}, b={freq:.2f}<br>"
            f"period≈{period:.3f}, $a^{{2}}/2$≈{est_energy:.3f}"
        )

    info(["<hr>", diagnostics_card], id="wave-diagnostics")
# params.snapshot() returns a symbol-keyed mapping; access values with the same symbols you registered.

## Using multiple tabs


In [ ]:
fig_tabs = Figure(x_range=(-6, 6), y_range=(-3, 3), sampling_points=400)
fig_tabs.add_view("phase", x_range=(-2 * pi, 2 * pi), y_range=(-3, 3), x_label="phase")
display(fig_tabs)

with fig_tabs:
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(b, value=0.5, min=-2.0, max=2.0, step=0.1)

    plot(a * sin(x) + b, x, id="shared-wave", view=("main", "phase"), color="crimson")
    with fig_tabs.view("phase"):
        plot(sp.cos(2 * x), x, id="phase-only", color="purple")

    info("<b>Shared card:</b> visible on every view", id="shared-card")
    info("<b>Phase note:</b> scoped to the phase tab", id="phase-card", view="phase")

# 2) Compendium: Frequently used operations

### Figure construction and ranges
- `Figure(...)`
- `get_x_range()`, `get_y_range()`
- `set_title(...)`

### Plot and parameter workflow
- `plot(...)`
- `parameter(symbol, ...)`
- `params[symbol]` / `params.snapshot()`
- `render()`

### Numeric expressions
- `numpify(expr, vars=...)`
- `DYNAMIC_PARAMETER` and `UNFREEZE`

### Symbolic parsing
- `parse_latex(latex_string)`


In [ ]:
# BUG: Compendium sucks. It should be much more explicit: description of functionality (quick) and common options.


### B) Symbolic-to-numeric with dynamic parameters

Use this when you want to start from symbolic math and still keep interactivity.


In [ ]:
expr = parse_latex(r"A \sin(k x + \phi)")
expr.free_symbols

In [ ]:
f = numpify(expr, vars=["x", "A", "k", "phi"])
f